In [3]:
from datasets import Dataset

# Hugging Face datasets 파일 읽기
train = Dataset.load_from_disk('/data/ephemeral/home/data/train_dataset/train')
dev = Dataset.load_from_disk('/data/ephemeral/home/data/train_dataset/validation')

# Pandas DataFrame으로 변환
train_df = train.to_pandas()
dev_df = dev.to_pandas()

dev_df.head(5)

,title,context,question,id,answers,document_id,__index_level_0__
0,전효숙,"순천여자고등학교 졸업, 1973년 이화여자대학교를 졸업하고 1975년 제17회 사법...",처음으로 부실 경영인에 대한 보상 선고를 받은 회사는?,mrc-0-003264,"{'answer_start': [284], 'text': ['한보철강']}",9027,2146
1,스토우빌선,요크 카운티 동쪽에 처음으로 여객 열차 운행이 시작한 시점은 1868년 토론토 & ...,스카버러 남쪽과 코보콘그 마을의 철도 노선이 처음 연장된 연도는?,mrc-0-004762,"{'answer_start': [146], 'text': ['1871년']}",51765,3106
2,촐라,"촐라 정부\n 촐라의 정부 체제는 전제군주제였으며,2001 촐라의 군주는 절대적인 ...",촌락에서 운영 위원 후보자 이름을 쓰기위해 사용된 것은?,mrc-1-001810,"{'answer_start': [517], 'text': ['나뭇잎']}",15694,1180
3,백조 기사,프랑스의 십자군 무훈시는 1099년 예루살렘 왕국의 통치자가 된 고드프루아 드 부용...,로타이르가 백조를 구하기 위해 사용한 것은?,mrc-1-000219,"{'answer_start': [1109], 'text': ['금대야']}",59536,141
4,탈관료제화,탈관료제화는 현대사회에서 관료제 성격이 약화되는 현상이다. 현대사회에서 관료제는 약...,의견을 자유롭게 나누는 것은 조직 내 어떤 관계에서 가능한가?,mrc-1-000285,"{'answer_start': [386], 'text': ['수평적 관계']}",32991,184


### answer text에 특정 단어가 포함되어있는지 확인

In [67]:
# 특정 단어 설정 (예시: "금대야")
keyword = "야호"

# answers의 text들 중 특정 단어가 포함되어 있는지 확인하고 결과를 출력하는 함수
def check_keyword_in_answers(df, keyword):
    found_any = False  # 해당 단어가 포함된 적이 있는지 확인하는 변수
    for index, row in df.iterrows():
        answer_texts = row['answers']['text']  # answers의 text 값
        for text in answer_texts:
            if keyword in text:
                print(f"'{keyword}'이(가) 포함된 text: {text}")
                found_any = True
    
    # 만약 해당 단어가 포함된 text가 하나도 없다면 메시지를 한 번만 출력
    if not found_any:
        print(f"'{keyword}'이(가) 포함되어 있지 않음")

# train_df에서 특정 단어가 포함된 answers의 text들을 확인
check_keyword_in_answers(train_df, keyword)


'\'이(가) 포함되어 있지 않음


### 1. 답변 길이 제약 설정

In [5]:
df = train_df # or dev_df

# text의 길이 계산 후 최대 및 최소 값 출력
text_lengths = df['answers'].apply(lambda x: len(x['text'][0]))
print(f"최대 길이: {text_lengths.max()}")
print(f"최소 길이: {text_lengths.min()}")
print('평균 길이:', sum(text_lengths)/len(text_lengths))

# 최대, 최소 길이 text 출력
print(f"최대 길이의 text: {df['answers'].apply(lambda x: x['text'][0])[text_lengths.idxmax()]}")
print(f"최소 길이의 text: {df['answers'].apply(lambda x: x['text'][0])[text_lengths.idxmin()]}")

최대 길이: 83
최소 길이: 1
평균 길이: 6.275050607287449
최대 길이의 text: “나는 국경일에 일장기를 게양하는 것을 반대하지 않는다. 왜냐하면 우리가 일본의 통치하에 있는 한 우리는 그 통치의 명령에 복종해야 하기 때문이다.”
최소 길이의 text: 덱


### 2. n-best 후보 필터링